In [113]:
import os
import numpy as np
from sklearn.datasets import load_diabetes
from PIL import Image
import matplotlib.pyplot as plt

In [114]:
class DenseLayer:
    """
    Represents a neural network layer.
    ==========
    Attributes:
    ----------
        size (int): Number of neurons in the layer.
        input_layer (bool): Whether the layer is an input layer.
        activation (str): Activation function for the layer.
        use_bias (bool): Whether to use bias in the layer.
        optimizer (str): Optimization algorithm used for the layer.
        w (numpy.ndarray): Weights matrix for the layer.

    Methods:
    ---------
        activationFunction(z):
            Apply the activation function to the given input.

        __call__(X):
            Perform a forward pass through the layer.

    """
    def __init__(
            self, 
            size, 
            *, 
            input_layer: bool = False,
            activation: str = "linear",
            use_bias: bool = True,
            ):
        """
        Initialize a neural network layer.

        Args:
            size (int): Count of neurons in the layer.
            input_layer (bool, optional): Whether the layer is an input layer. Defaults to False.
            activation (str, optional): Activation function for the layer. Can be "linear", "relu", or "sigmoid". Defaults to "linear".
            use_bias (bool, optional): Whether to use bias in the layer. Defaults to True.
        """
            
        
        self.size = size
        self.input_layer = input_layer
        self.activation = activation
        self.use_bias = use_bias

        self.optimizer = None # Optimizer for layer

        self._input = None
        self._output = None

        self.w = None # Weights matrix
        self._weight_gradient = None # Weights derivative matrix
        self._bias_gradient = None # Biases derivative vector

    def _weightInit(self, input_size):
        """
        Initialize the weights matrix based on the input size.

        Args:
            input_size (int): Size of the input.

        Notes:
            Only executed for layers other than the input layer.
        """

        if self.input_layer:
            return # input_layer doesn't need weights

        self.w = np.random.normal(loc = 0, scale = 1 / input_size, size=(input_size, self.size))
        # Initialize weights matrix using a normal distribution with mean 0 and variance 1 / input_size

        self.bias = np.zeros((1, self.size))
        # Initialize biases as zeros

    def activationFunction(self, z):
        """
        Apply the activation function to the given input.

        Args:
            z (numpy.ndarray): Input to the activation function.

        Returns:
            numpy.ndarray: Output after applying the activation function.
        """

        if self.activation == "linear":
            return z

        if self.activation == "relu":
            return np.maximum(z, np.zeros(z.shape))

        if self.activation == "sigmoid":
            return 1 / (1 + np.exp(-z))

    def _setOptimizer(self, optimizer, beta_1, beta_2):
        """
        Set the optimizer and initialize optimizer-specific variables.

        Args:
            optimizer (str): Optimization algorithm to use.
            beta_1 (float): Value for the optimizer parameter beta_1.
            beta_2 (float): Value for the optimizer parameter beta_2.

        Notes:
            - Only executed for layers other than the input layer.
            - Sets the optimizer and initializes optimizer-specific variables based on the chosen optimizer.
            - For each optimizer, the corresponding variables are initialized.
        """

        if self.input_layer:
            return

        self.optimizer = optimizer
        self._b1 = beta_1
        self._b2 = beta_2

        if self.optimizer == "adagrad":
            self._weight_v = np.zeros(self.w.shape)
            # Initialize weight-specific variables for AdaGrad

            if self.use_bias:
                self._bias_v = np.zeros(self.bias.shape)
                # Initialize bias-specific variables for AdaGrad

        if self.optimizer == 'adam':
            self._iter = 0  # Calculate iterations

            self._weight_m = np.zeros(self.w.shape)
            self._weight_v = np.zeros(self.w.shape)
            # Initialize weight-specific variables for Adam

            if self.use_bias:
                self._bias_m = np.zeros(self.bias.shape)
                self._bias_v = np.zeros(self.bias.shape)
                # Initialize bias-specific variables for Adam

        if self.optimizer == 'rms_prop':
            self._weight_v = np.zeros(self.w.shape)

            if self.use_bias:
                self._bias_v = np.zeros(self.bias.shape)
            # Initialize weight and bias-specific variables for RMSprop

        if self.optimizer == 'gdm':
            self._weight_m = np.zeros(self.w.shape)

            if self.use_bias:
                self._bias_m = np.zeros(self.bias.shape)
            # Initialize weight and bias-specific variables for Gradient Descent with Momentum   

    def _activationDerivative(self):
        """
        Compute the derivative of the activation function.

        Returns:
            numpy.ndarray: Derivative of the activation function.

        Notes:
            Only supports the "linear", "relu", and "sigmoid" activation functions.
        """

        if self.activation == "linear":
            return 1

        if self.activation == "relu":
            return (self._output > 0) * 1

        if self.activation == "sigmoid":
            return self._output * (1 - self._output)

    def _setGrad(self, grad):
        """
        Calculate the gradients of weights and bias for backpropagation.

        Args:
            grad (numpy.ndarray): Gradient from the previous layer.

        Returns:
            numpy.ndarray: Gradient to be passed to the previous layer.

        Notes:
            Only executed for layers other than the input layer.
        """

        if self.input_layer:
            return
        
        grad = grad * self._activationDerivative()
        self._weight_gradient = self._input.T @ grad

        if self.use_bias:
            self._bias_gradient = grad.sum(axis=0, keepdims=True)

        return grad @ self.w.T
    
    def _updateGrad(self, learning_rate):
        """
        Update the weights and bias based on the computed gradients.

        Args:
            learning_rate (float): Learning rate for gradient descent.

        Notes:
            - Only executed for layers other than the input layer.
            - Updates the weights and biases based on the computed gradients and the chosen optimizer.
            - For each optimizer, the corresponding update rule is applied.
        """



        if self.input_layer:
            return

        eps = 10e-8 # Optimizer's epsilon

        if self.optimizer == "gd":
            self.w -= learning_rate * self._weight_gradient
            if self.use_bias:
                self.bias -= learning_rate * self._bias_gradient

        if self.optimizer == "adagrad":
            self._weight_v += np.square(self._weight_gradient)
            learning_rate_weight = learning_rate / ( np.sqrt(self._weight_v) + eps)

            self.w -= learning_rate_weight * self._weight_gradient

            if self.use_bias:
                self._bias_v += np.square(self._bias_gradient)
                learning_rate_bias = learning_rate / ( np.sqrt(self._bias_v) + eps)

                self.bias -= learning_rate_bias * self._bias_gradient

        if self.optimizer == 'adam':
            self._iter += 1

            self._weight_m = self._b1 * self._weight_m + (1- self._b1) * self._weight_gradient
            self._weight_v = self._b2 * self._weight_v + (1- self._b2) * np.square(self._weight_gradient)

            weight_m = self._weight_m / (1 - np.power(self._b1, self._iter))
            weight_v = self._weight_v / (1 - np.power(self._b2, self._iter))

            self.w -= learning_rate * weight_m / (np.sqrt(weight_v) + eps) # Updating

            if self.use_bias:
                self._bias_m = self._b1 * self._bias_m + (1- self._b1) * self._bias_gradient
                self._bias_v = self._b2 * self._bias_v + (1- self._b2) * np.square(self._bias_gradient)

                bias_m = self._bias_m / (1 - np.power(self._b1, self._iter)) 
                bias_v = self._bias_v / (1 - np.power(self._b2, self._iter))


                self.bias -= learning_rate * bias_m / (np.sqrt(bias_v) + eps) # Updating

        
        if self.optimizer == 'rms_prop':
            self._weight_v = self._b2 * self._weight_v + (1- self._b2) * np.square(self._weight_gradient)

            learning_rate_weight = learning_rate / ( np.sqrt(self._weight_v) + eps)

            self.w -= learning_rate_weight * self._weight_gradient

            if self.use_bias:
                self._bias_v = self._b2 * self._bias_v + (1- self._b2) * np.square(self._bias_gradient)
                learning_rate_bias = learning_rate / ( np.sqrt(self._bias_v) + eps)

                self.bias -= learning_rate_bias * self._bias_gradient

        if self.optimizer == 'gdm':
            self._weight_m = self._b2 * self._weight_m + (1 - self._b2) * self._weight_gradient

            self.w -= learning_rate * self._weight_m

            if self.use_bias:
                self._bias_m = self._b2 * self._bias_m + (1 - self._b2) * self._bias_gradient

                self.bias -= learning_rate * self._bias_m

    def __call__(self, X):
        """
        Perform a forward pass through the layer.

        Args:
            X (numpy.ndarray): Input to the layer.

        Returns:
            numpy.ndarray: Output of the layer after applying the activation function.
        """
        if self.input_layer:
            return X
        
        self._input = X
        self._output = self.activationFunction(X @ self.w + self.bias)

        return self._output

In [131]:
class Conv2d:
    def __init__(
            self,
            size: int,
            kernel_size: tuple,
            *,
            padding: int = 0,
            stride: int = 1,
            activation: str = "linear",
            mode: str = "rgb",
            global_pooling: str = None,
            use_bias: bool = True,
        ):
        self.size = size
        self.kernel_size = kernel_size

        self.padding = padding
        self.stride = stride
        self.activation = activation
        self.mode = mode
        self.global_pooling = global_pooling
        self.use_bias = use_bias

        self.kernel = None
        self.optimizer = None

        self._kernel_gradient = None # Kernel derivative matrix

        if self.use_bias:
            self._bias_gradient = None # Biases derivative vector

    def _weightInit(self, depth):
        self.kernel_size = self.kernel_size + (depth, ) # (kenel_size, depth)

        # Initialize weights matrix using a normal distribution with mean 0 and variance 1 / input_size
        self.kernel = np.random.normal(loc = 0, scale = 1 / (self.kernel_size[0] * self.kernel_size[1] * self.kernel_size[2]), size=(self.kernel_size + (self.size,)))        

        # Description here:
        self._kernel_gradient = np.zeros_like(self.kernel)

        if self.use_bias:
            # Initialize biases as zeros
            self.bias = np.zeros((self.size, 1))

            # Description here:
            self._bias_gradient = np.zeros_like(self.bias)

    def activationFunction(self, z):
        """
        Apply the activation function to the given input.

        Args:
            z (numpy.ndarray): Input to the activation function.

        Returns:
            numpy.ndarray: Output after applying the activation function.
        """

        if self.activation == "linear":
            return z

        if self.activation == "relu":
            return np.maximum(z, np.zeros(z.shape))
        
        if self.activation == "sigmoid":
            return 1 / (1 + np.exp(-z))

    def _activationDerivative(self):
        """
        Compute the derivative of the activation function.

        Returns:
            numpy.ndarray: Derivative of the activation function.

        Notes:
            Only supports the "linear", "relu", and "sigmoid" activation functions.
        """

        if self.activation == "linear":
            return 1

        if self.activation == "relu":
            return (self._output > 0) * 1

        if self.activation == "sigmoid":
            return self._output * (1 - self._output)

    def _setOptimizer(self, optimizer, beta_1, beta_2):
        self.optimizer = optimizer
        self._b1 = beta_1
        self._b2 = beta_2

        # WILL BE DELETED

    def _setGrad(self, grad):
        if self.global_pooling == "average":
            grad = grad.T[np.newaxis, np.newaxis, ...]
            output_gradient = np.ones(self.output_shape) / (self.output_shape[0] * self.output_shape[1])
        else:
            output_gradient = np.ones(self.output_shape)

        if self.use_bias:
            self._bias_gradient = (grad * self._activationDerivative() * output_gradient).reshape(-1, grad.shape[-2]).sum(axis=0, keepdims=True).T 
        
        output_gradient = output_gradient * self._activationDerivative() * grad

        self._input_gradient = np.ones_like(self._input)

        self._kernel_gradient = np.zeros_like(self.kernel)

        
        for index in range(self._input.shape[-1]):
            for i in range(len(self._indices_axis1)):
                    x_1, x_2 = self._indices_axis1[i]
                    for j in range(len(self._indices_axis2)):
                        y_1, y_2 = self._indices_axis2[j]

                        self._kernel_gradient += self._input[x_1:x_2, y_1:y_2, :, [index]] * output_gradient[i, j, :, [index]]
                        self._input_gradient[x_1:x_2, y_1:y_2, :, [index]] = self._input_gradient[x_1:x_2, y_1:y_2, :, [index]] * (self.kernel * output_gradient[i, j, :, [index]]).sum(axis=3, keepdims=True)

        return self._input_gradient

    def _updateGrad(self, learning_rate):
        eps = 10e-8 # Optimizer's epsilon

        self.optimizer == "gd":
        self.kernel -= learning_rate * self._kernel_gradient

        if self.use_bias:
            self.bias -= learning_rate * self._bias_gradient

    def __call__(self, tensor):
        self._input = tensor

        tensor_shape = np.array(tensor.shape)
        feature_map_shape = ((tensor_shape[:2] + 2 * self.padding - self.kernel_size[:2]) / self.stride).astype(int) + 1
        self.output_shape = np.concatenate([feature_map_shape, [self.size, tensor_shape[-1]]])

        self._output = np.zeros(self.output_shape)

        self._indices_axis1 = [(i - self.kernel_size[0], i) for i in range(self.kernel_size[0], tensor_shape[0] + 1, self.stride)]
        self._indices_axis2 = [(i - self.kernel_size[1], i) for i in range(self.kernel_size[1], tensor_shape[1] + 1, self.stride)]

        for index in range(tensor_shape[-1]):
            for i in range(len(self._indices_axis1)):
                    x_1, x_2 = self._indices_axis1[i]
                    for j in range(len(self._indices_axis2)):
                        y_1, y_2 = self._indices_axis2[j]
                        self._output[i, j, :, index] = (self._input[x_1:x_2, y_1:y_2, :, [index]] * self.kernel).sum(axis=(0, 1, 2))
        
        activated_output = self.activationFunction(self._output) + self.bias

        if self.global_pooling == "average":
            return activated_output.mean(axis=(0, 1)).T

        # Can be added Max pooling

        return activated_output

In [118]:
# image = Image.open("image.jpeg")
# image_array = np.array(image)[..., np.newaxis]

# layer = Conv2d(7, (5, 5), stride=2)
# layer._weightInit(3)
# output1 = layer(X)

# layer2 = Conv2d(4, (3, 3), stride=3, global_pooling="max")
# layer2._weightInit(7)
# output2 = layer2(output1)

In [119]:
# plt.imshow(output1[:, :, 2, 120])

In [130]:
class NeauralNetwork:

    def __init__(
            self, 
            layers: list, 
            loss_function: str = "mse", 
            softmax: bool = False,
            learning_rate = 0.01,
            verbose: bool = False,
            optimizer: str = "gd",
            epochs: int = 1, 
            batch_size: int = 32,
            beta_1: float = 0.9,
            beta_2: float = 0.999
            ):
        self.layers = layers
        self.loss_function = loss_function
        self.softmax = softmax
        self.learning_rate = learning_rate
        self.verbose = verbose
        self.optimizer = optimizer  # Optimizer for all layers
        self.epochs = epochs
        self.batch_size = batch_size

        self.beta_1 = beta_1  # Optimizer parameters
        self.beta_2 = beta_2  # Optimizer parameters


        # Weights initializing:
        self.layers[0]._weightInit(3) # TODO: KEEP ATTENTION MAY BE CHANGED
        self.layers[0]._setOptimizer(self.optimizer, self.beta_1, self.beta_2)

        for i in range(1, len(self.layers)):
            self.layers[i]._weightInit(self.layers[i - 1].size)
            self.layers[i]._setOptimizer(self.optimizer, self.beta_1, self.beta_2)
            # Initialize weights for each layer and set the optimizer

    def lossFunction(self, y_true, y_pred):

        if self.loss_function == "mse":
            return 0.5 * np.mean(np.linalg.norm(y_pred - y_true, axis=1)**2)
        
        if self.loss_function == "cross_entropy":
            self.probabilities_ = np.exp(y_pred)
            self.probabilities_ = self.probabilities_ / np.sum(self.probabilities_, axis=1, keepdims=True)

            return -(np.log(self.probabilities_[np.arange(y_true.shape[0]), np.argmax(y_true, axis=1)])).sum()

        # Can be added

    def _lossFunctionDerivative(self, y_pred, y_true):
        if self.loss_function == "mse":
            derivative = 1 / len(y_pred) * (y_pred - y_true)
        
        if self.loss_function == "cross_entropy":
            return self.probabilities_ - y_true
        
        return derivative

    def fit(self, X, y):

        batch_separation = [(i, i + self.batch_size) for i in range(0, X.shape[-1], self.batch_size)] # Get batch indices
        epoch_len = len(batch_separation)

        indices = np.arange(X.shape[-1])

        for _ in range(self.epochs):
            np.random.shuffle(indices) # Shuffle the training data

            for iter, (i, j) in enumerate(batch_separation):
                X_ = X[:, :, :, indices[i:j]] # Get current batch
                y_ = y[indices[i:j]] # Get current batch

                pred = self.forward(X_)

                if self.verbose:
                    process_percent = int(iter / epoch_len * 10)
                    print(f"\r Epoch {_ + 1}/{self.epochs}; Batch {iter}/{epoch_len}: [{process_percent * '=' + '>' + (10 - process_percent) * '-'}] - loss: {self.lossFunction(y_, pred)}",end='')
                
                self.backward(pred, y_)
            
            if self.verbose:
                print(f"\r Epoch {_ + 1}/{self.epochs}; Batch {iter + 1}/{epoch_len}: [{11 * '='}] - loss: {self.lossFunction(y_, pred)}")

    def predict(self, X):
        return self.forward(X)
        
    def forward(self, X):
        X_ = np.copy(X)
        
        for layer in self.layers:
            X_ = layer(X_)

        return X_

    def backward(self, y_pred, y_true):   
        gradient = self._lossFunctionDerivative(y_pred, y_true)

        for layer in reversed(self.layers):
            gradient = layer._setGrad(gradient)
            layer._updateGrad(self.learning_rate)

In [122]:
def makeData(path, labels):
    X = np.concatenate([np.asarray(Image.open(path + image_path))[..., np.newaxis] / 256 for image_path in os.listdir(path)], axis=3)

    # for image_path in os.listdir(path):
    #     image = Image.open(path + image_path)
    #     image_array = np.asarray(image) / 256

    #     label = np.zeros(3)
    #     label[[labels[image_path.split("_")[0]]]] = 1

    #     Dataset.append((image_array, label))

    return X

labels = {
    "cucumber": 0,
    "eggplant": 1,
    "mushroom": 2,
}

path = "./data/train/"

X = makeData(path, labels)   

In [123]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
# ohe.fit(os.listdir(path))

y_cat = np.array([label.split('_')[0] for label in os.listdir(path)]).reshape(-1, 1)

y = ohe.fit_transform(y_cat)

In [124]:
np.random.random((3, 3, 10, 80)).sum(axis=3, keepdims=True).shape

(3, 3, 10, 1)

In [132]:
nn = NeauralNetwork(layers=[
        Conv2d(5, (23, 23), stride=16, activation='relu'),
        Conv2d(10, (7, 7), stride=2, activation='relu', global_pooling="average"),

        DenseLayer(size=20, activation="relu"),
        DenseLayer(size=3),
    ],
    loss_function = "cross_entropy",
    learning_rate=0.1,
    softmax=True,
    verbose=True,
    optimizer="gd",
    batch_size=150,
    epochs=10,
)

nn.fit(X, y)

 Epoch 1/10; Batch 0/1: [>----------] - loss: 164.79255935422967[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
 Epoch 1/10; Batch 1/1: [===========] - loss: 164.79255935422967
 Epoch 2/10; Batch 0/1: [>----------] - loss: 165.11178030257548[[ 0.01455999]
 [-0.00706318]
 [-0.00490672]
 [ 0.00335552]
 [ 0.002504  ]
 [-0.00827753]
 [ 0.00190723]
 [ 0.00180504]
 [ 0.00341407]
 [-0.00918063]]
[[-4.14889771e-05]
 [-1.81735604e-05]
 [ 3.41796210e-05]
 [-2.50128048e-05]
 [ 5.17382559e-05]]
 Epoch 2/10; Batch 1/1: [===========] - loss: 165.11178030257548
 Epoch 3/10; Batch 0/1: [>----------] - loss: 170.22896749939338[[ 0.01286788]
 [-0.02403571]
 [-0.02608284]
 [ 0.01901677]
 [-0.00247603]
 [-0.01421207]
 [-0.00234772]
 [ 0.04382881]
 [ 0.02089852]
 [ 0.01940092]]
[[-6.87207663e-05]
 [-3.24617510e-05]
 [ 8.09992405e-05]
 [ 2.44388926e-05]
 [ 8.68020257e-05]]
 Epoch 3/10; Batch 1/1: [===========] - loss: 170.22896749939338
 Epoch 4/10; Batch 0/1: [>-

KeyboardInterrupt: 